In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

from torch.autograd import Variable

! pip install tensorly
import tensorly as tl
from torch.autograd import Variable
tl.set_backend('pytorch')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
from fastai.basics import *

In [ ]:
import tqdm
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
train_dataset = datasets.MNIST('classifier_data', train=True, download=True)
train_indices = (train_dataset.targets == 3) | (train_dataset.targets == 7)
data, targets = train_dataset.data[train_indices], train_dataset.targets[train_indices]
# print(data)
# print(targets)

100%|██████████| 9912422/9912422 [00:00<00:00, 75963733.98it/s]


Extracting classifier_data/MNIST/raw/train-images-idx3-ubyte.gz to classifier_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 78253032.19it/s]

Extracting classifier_data/MNIST/raw/train-labels-idx1-ubyte.gz to classifier_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25725795.75it/s]

Extracting classifier_data/MNIST/raw/t10k-images-idx3-ubyte.gz to classifier_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4522917.56it/s]

Extracting classifier_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to classifier_data/MNIST/raw



In [ ]:
#QIKSIT TRAINING

!pip install qiskit
!pip install qiskit_machine_learning
from qiskit import QuantumCircuit, assemble, Aer
from qiskit_machine_learning.algorithms.classifiers import VQC
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2, TwoLocal, NLocal
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, ADAM

In [ ]:
def FeatureMap():
  num_qubits = 4
  x = ParameterVector('x', length = num_qubits)
  feature_map = QuantumCircuit(num_qubits)

  for i in range(num_qubits):
    feature_map.rx(x[i],i)
    feature_map.h(i)
  return feature_map

In [ ]:
from qiskit.circuit.library.n_local import real_amplitudes
def VarCirc():
  num_qubits = 4
  #var_circuit = TwoLocal(4, ['rx', 'ry','rz'], 'cx', entanglement = 'circular', reps = 1, insert_barriers = True)
  reps = 3

  psi = ParameterVector('psi', length = num_qubits*reps*3)
  var_circuit = QuantumCircuit(num_qubits)

  for j in range(reps):
    for i in range(num_qubits-1):
      var_circuit.cx(i,i+1)
    var_circuit.cx(num_qubits-1, 0)

    for i in range(num_qubits):
      var_circuit.rx(psi[3*i + 0 + num_qubits*3*j], i)
      var_circuit.ry(psi[3*i + 1 + num_qubits*3*j], i)
      var_circuit.rz(psi[3*i + 2 + num_qubits*3*j], i)

  return var_circuit

In [ ]:
import random
subset_i = random.sample(range(len(data)), 100)
data = data[subset_i]
targets = targets[subset_i]

In [ ]:
data = data.reshape(len(data), 784)
targets = np.array([int(i==7) for i in targets])
# print(data)
# print(targets)

In [ ]:
import torch.nn as nn
import math
class TNLayer(nn.Module):
    def __init__(self):
        super(TNLayer, self).__init__()
        # Create the variables for the layer.
        self.a_var = nn.Parameter(torch.randn(28, 28, 2) * (1.0 / 28.0))
        self.b_var = nn.Parameter(torch.randn(28, 28, 2) * (1.0 / 28.0))
        self.bias = nn.Parameter(torch.zeros(28, 28))

    def forward(self, inputs):
        # Define the contraction.
        def f(input_vec, a_var, b_var, bias_var):
            # Reshape to a matrix instead of a vector.
            input_vec = input_vec.reshape(28, 28)
             #input_vec = input_vec.reshape(2, 512)

            # Now we create the network.
            a = a_var.unsqueeze(0)  # Shape: (1, 32, 32, 2)
            b = b_var.unsqueeze(0)  # Shape: (1, 32, 32, 2)
            x_node = input_vec.unsqueeze(-1)  # Shape: (32, 32, 1)

            # Contract a with x_node
            c = torch.einsum("ijkl,ijm->ijkm", a, x_node)  # Shape: (32, 32, 32, 2)

            # Contract the result with b
            result = torch.einsum("ijkl,ijkl->ij", c, b)  # Shape: (32, 32)

            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, loop through the input batch.
        results = []
        for input_vec in inputs:
            results.append(f(input_vec, self.a_var, self.b_var, self.bias))
        return nn.functional.relu(torch.stack(results))



class VQC(nn.Module):
  def __init__(self, n, num_layers):
    super(VQC, self).__init__()
    self.n = n
    self.num_layers = num_layers
    self.params = nn.Parameter(torch.randn(self.n, 3 * self.num_layers))

  def prints(self, psi):
    print("-------------------------MPS STRUCTURE SHAPE------------------------------------------")
    for _, i in enumerate(psi):
      print(i.shape)
    print("-------------------------SHAPE------------------------------------------")

  def H(self, i, psi):
    H_matrix = 1 / ((2) ** 0.5) * torch.tensor([[1, 1],
                                                 [1, -1]], dtype=torch.cfloat)
    psi[i] = torch.tensordot(H_matrix, psi[i], ([1], [2]))
    psi[i] = torch.moveaxis(psi[i], 0, 2)
    return psi

  def RX(self, i, j, psi):  # Alter the array psi into the array where H has acted on the ith qubit
    t1 = (torch.cos(i / 2)).reshape(1)
    t2=(torch.sin(i/2)*-1j).reshape(1)
    t3=(torch.sin(i/2)*-1j).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rx=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rx,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi
  def RY(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.cos(i/2)).reshape(1)
    t2=-1.*(torch.sin(i/2)).reshape(1)
    t3=(torch.sin(i/2)).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    ry=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(ry,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def RZ(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.exp(-1j*i/2)).reshape(1)
    t2=(torch.zeros(1)).reshape(1)
    t3=(torch.zeros(1)).reshape(1)
    t4=(torch.exp(1j*i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rz=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rz,psi[j],([1],[2]))
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def CNOT(self, i, j, psi):
     dim = psi[i].shape[0]
     mps=torch.tensordot(psi[i],psi[j],([1],[0]))
     mps=torch.moveaxis(mps,2,1)
     CNOT_matrix=torch.tensor([[1,0,0,0],
                       [0,1,0,0],
                       [0,0,0,1],
                       [0,0,1,0]],dtype=torch.cfloat)
     CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
     mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2, 3]))
     mps=torch.moveaxis(mps,1,3).reshape((2*dim,2*dim))
     #mps+=mps+(torch.randn(mps.shape))*0.001
     data=torch.rand(mps.shape)
     data=data*mps.abs().max()*0.000001/data.max()
     mps=mps+data
     #u,s,v = torch.linalg.svd(mps)
     u,s,v = torch.svd(mps)
     v = torch.conj(v.T)
     c = torch.diag(s[:dim])
     mps1 = torch.mm(u[:,:dim], c.type(torch.cfloat))
     mps1 = mps1.reshape((2,dim, dim))
     psi[i] = torch.moveaxis(mps1,0,2)
     mps2 = v[:dim,:]
     psi[j] = mps2.reshape((dim, dim,2))
     return psi

  def CNOT_full(self,i,j,psi):
        # Alter the array psi into the array where H has acted on the ith qubit
        #i-->control;
        #j-->target;
        s0=psi[i].shape[0]
        s1=psi[j].shape[1]
        # print(i,j)
        # print(psi[i].shape)
        # print(psi[j].shape)
        mps=torch.tensordot(psi[i],psi[j],([1],[0]))
        mps=torch.moveaxis(mps,2,1)
        #print(mps.shape)
        CNOT_matrix=torch.tensor([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]],dtype=torch.cfloat)

        CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
        mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2,3]))
        mps=torch.moveaxis(mps,1,3).reshape((s0*2,s1*2))
        data=torch.rand(mps.shape)
        data=data*mps.abs().max()*0.00001/data.max()
        mps=mps+data
        u, s, v =  torch.linalg.svd(mps)
        c=(torch.zeros(u.shape[1],v.shape[0]))
        for h,k in enumerate(s):
          c[h,h]=k
        mps1=torch.mm(u, c.type(torch.cfloat))
        mps1=mps1.reshape((2,int(mps1.shape[0]/2),mps1.shape[1]))
        psi[i]=torch.moveaxis(mps1,0,2)
        mps2=v#.transpose(-2, -1)
        psi[j]=mps2.reshape((mps2.shape[0],int(mps2.shape[1]/2),2))
        return psi

  def getParams(self):
    return self.params

  def setParams(self, params_matrix):
    self.params = params_matrix

  def compute_tensor_circuit(self, psi, t):
    #Embedding layer
    for i in range(self.n):
      self.RX(t[i], i, psi)
      self.H(i, psi)

    # Parametrized Layers
    #Each layer contains a series of parametrized RX, RY, RZ gates and cyclic entanglement
    for i in range(self.num_layers):
      #CNOTs
      for j in range(self.n-1):
        psi = self.CNOT(j, j+1, psi)
      psi = self.CNOT(self.n-1,0, psi)

      #Parametrized rotations
      for j in range(self.n):
        psi =self.RX(self.params[j][3*i + 0], j, psi)
        psi = self.RY(self.params[j][3*i + 1], j, psi)
        psi = self.RZ(self.params[j][3*i + 2], j, psi)

    return psi

  def contract_tensor_ring(self,psi):
    psi_new = psi[0]
    for i in range(self.n-2):
      psi_new = torch.tensordot(psi_new, psi[i+1], ([1], [0]))
      psi_new = torch.moveaxis(psi_new, -2, 1)

    psi_new = torch.tensordot(psi_new, psi[self.n-1], ([0,1], [1,0]))
    return psi_new

  # def FeatureMap():
  #   num_qubits = 4
  #   x = ParameterVector('x', length = num_qubits)
  #   feature_map = QuantumCircuit(num_qubits)

  #   for i in range(num_qubits):
  #      feature_map.rx(x[i],i)
  #      feature_map.h(i)
  #   return feature_map

  # def VarCirc():
  #    num_qubits = 4
  # #var_circuit = TwoLocal(4, ['rx', 'ry','rz'], 'cx', entanglement = 'circular', reps = 1, insert_barriers = True)
  #    reps = 3

  #    psi = ParameterVector('psi', length = num_qubits*reps*3)
  #    var_circuit = QuantumCircuit(num_qubits)

  #    for j in range(reps):
  #     for i in range(num_qubits-1):
  #        var_circuit.cx(i,i+1)
  #        var_circuit.cx(num_qubits-1, 0)

  #    for i in range(num_qubits):
  #     var_circuit.rx(psi[3*i + 0 + num_qubits*3*j], i)
  #     var_circuit.ry(psi[3*i + 1 + num_qubits*3*j], i)
  #     var_circuit.rz(psi[3*i + 2 + num_qubits*3*j], i)

  #    return var_circuit


  def forward(self, psi, t):

    # Convert tensor t to complex tensor
    # print(t.size())
    # t_complex = torch.view_as_complex(t)
    # print(t_complex.size())
    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    k1 = k[0, 0, 0, 0].abs().reshape(1) ** 2
    k2 = k[0, 0, 1, 0].abs().reshape(1) ** 2
    k3 = k[1, 1, 1, 1].abs().reshape(1) ** 2

    return torch.cat([k1, k2, k3]).reshape(1, 3)

  def final_state(self, psi,t):

    #psi_final = self.test_circuit(psi,t)
    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    return k

  def create_zero_tensor(num_qubits, d):
    psi_0 = []
    for j in range(num_qubits):
      k = torch.zeros((d,d,2), dtype=torch.cfloat)
      k[0,0,0] = 1.0
      psi_0.append(k)
    return psi_0



In [ ]:
from qiskit.utils import QuantumInstance, algorithm_globals
backend = Aer.get_backend('aer_simulator_matrix_product_state')
backend_options = {"method": "statevector"}
seed = 987
quantum_instance = QuantumInstance(backend, shots=1024,seed_simulator = seed, seed_transpiler = seed,
                                   backend_options=backend_options)

<ipython-input-11-3721013baf0a>:5: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024,seed_simulator = seed, seed_transpiler = seed,


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# num_qubits = 4
# X = StandardScaler().fit_transform(data)
# pca = PCA(num_qubits)
# X = pca.fit_transform(X)

# scaler = MinMaxScaler((-3.142, 3.142))
# X_scaled = scaler.fit_transform(X)
# X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(
# X_scaled, targets, test_size=0.25, random_state=2)

In [ ]:
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()
        self.tn_layer = TNLayer()
        self.vqc = VQC(4, 2)
        #self.vqc = VQC(feature_map=FeatureMap(), ansatz=VarCirc(), loss="cross_entropy", optimizer= COBYLA(maxiter=50), quantum_instance=quantum_instance,)
        #self.output_layer = nn.Linear(2, 2)  # Adjust the output layer to match the number of classes

    def forward(self, x):
        x = self.tn_layer(x)
        #print(x.shape)
        x = x.reshape(len(x), 784)
        #print(x.shape)
        #print("++++++")
        #print(x)
        scaler = MinMaxScaler((-3.142, 3.142))
        y = StandardScaler().fit_transform(x)
        pca = PCA(4)
        X1 = pca.fit_transform(y)
        X_scaled = scaler.fit_transform(X1)
        X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_scaled, targets, test_size=0.25, random_state=2)
        #print(X1.shape)
        print("++++++")
        #print(X1[0])
        Y1 =[]
        for i in range(100):
        #X1 = self.vqc(VQC.create_zero_tensor(4, 2), X1.view(X1.shape[0], -1))
           Y1.append(self.vqc(VQC.create_zero_tensor(4, 2), torch.tensor(X1[i])))
        #X1 = self.vqc(VQC.create_zero_tensor(4, 2), X1)
        # feature_map = FeatureMap()
        # variational_circuit = VarCirc()
        #X1 = VQC(feature_map=FeatureMap(), ansatz=VarCirc(), loss="cross_entropy", optimizer= COBYLA(maxiter=50), quantum_instance=quantum_instance,)
    #optimizer = ADAM(maxiter=30, lr=1e-3, tol = 1e-6),
        #X1 = self.output_layer(X1)
        #print(Y1)
        #print(Y1.shape)
        Y1=torch.stack(Y1)
        #print(Y1.shape)
        return Y1

In [ ]:
# Create the neural network
net = MyNeuralNetwork()
# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=0.001)

# Training the neural network
num_epochs = 50
for epoch in range(num_epochs):
  with torch.no_grad():
    optimizer.zero_grad()
    outputs = net(data)
    # print(outputs)
    # print(outputs.shape)
    #print(labels.shape)
    Z=torch.squeeze(outputs, 1)
    #Z = Z.type(torch.int32)
    # print(Z)
    # print(Z.shape)
    #T = torch.targets(100, 1,3)
    #torch.reshape(torch.targets, (100, 1, 3))
    targets=torch.tensor(targets)
    # print(targets)
    # print(targets.shape)
    loss = criterion(Z, targets)
    loss = Variable(loss, requires_grad = True)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
    #loss_list_train.append(loss.clone())
    #accuracy_list_train.append(accuracy/100)

In [ ]:
def create_zero_tensor(num_qubits, d):
  psi_0 = []
  for j in range(num_qubits):
    k = torch.zeros((d,d,2), dtype=torch.cfloat)
    k[0,0,0] = 1.0
    psi_0.append(k)
  return psi_0

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

iris = load_iris()
X = iris['data']
y = iris['target']
#X=X[y!=2]
#y=y[y!=2]
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1
# which is importance for convergence of the neural network
#scaler = StandardScaler()
scaler = MinMaxScaler((-3.142, 3.142))
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(
    X_scaled, y, test_size=0.25, random_state=2)

In [ ]:
import tqdm
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
num_qubits = 4
num_layers = 1
bond = 16

In [ ]:
model_test = VQC(num_qubits,num_layers)
optimizer = torch.optim.Adam(model_test.parameters(), lr = 0.01, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()
EPOCHS = 25
X_train = (torch.from_numpy(X_train_0)).float()
y_train = (torch.from_numpy(y_train_0)).long().reshape(X_train.shape[0],1)
X_test  = Variable(torch.from_numpy(X_test_0)).float()
y_test  = Variable(torch.from_numpy(y_test_0)).long().reshape(X_test.shape[0],1)
model_test.train()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))
accuracy_list_train = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
  l = 0
  c = []
  for b,s in enumerate(X_train):
    psi = create_zero_tensor(num_qubits, bond)
    k1 = F.softmax(model_test(psi,s), dim = 1)
    loss = F.cross_entropy(k1, y_train[b])
    correct = (torch.argmax(k1, dim=1) == y_train[b]).type(torch.FloatTensor)
    c.append(correct.item())
    l+=loss/4
    if int((b+1)%4)==0:
      optimizer.zero_grad()
      l.backward(create_graph = True)
      optimizer.step()
      loss_list[epoch] = l.item()
      loss_epoch = float(l)
      l=torch.zeros(1)
    accuracy_list_train[epoch] = np.array(c).mean()
  with torch.no_grad():
    c=[]
    for j,s in enumerate(X_test):
      psi = create_zero_tensor(num_qubits, bond)
      k1 = F.softmax(model_test(psi,s), dim = 1)
      correct = (torch.argmax(k1, dim=1) == y_test[j]).type(torch.FloatTensor)
      c.append(correct.item())
    accuracy_list[epoch] = np.array(c).mean()
  print("Train Loss: {0:1.4f}; Train Accuracy: {1:1.4f}; Test Accuracy: {2:1.4f}".format(loss_epoch, accuracy_list_train[epoch], accuracy_list[epoch]))


  0%|          | 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1151.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  4%|▍         | 1/25 [00:02<00:51,  2.17s/it]

Train Loss: 1.0919; Train Accuracy: 0.4375; Test Accuracy: 0.6316


  8%|▊         | 2/25 [00:04<00:48,  2.13s/it]

Train Loss: 1.0891; Train Accuracy: 0.4464; Test Accuracy: 0.6579


 12%|█▏        | 3/25 [00:06<00:46,  2.10s/it]

...

Train Loss: 1.0615; Train Accuracy: 0.5268; Test Accuracy: 0.5000


 96%|█████████▌| 24/25 [01:00<00:02,  2.58s/it]

Train Loss: 1.0571; Train Accuracy: 0.5804; Test Accuracy: 0.5789


100%|██████████| 25/25 [01:02<00:00,  2.52s/it]

Train Loss: 1.0516; Train Accuracy: 0.5714; Test Accuracy: 0.6053
